In [4]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import io
from random import shuffle
import shutil
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

### parse xml to df

In [8]:
annot_path='/home/silas/MIDS/W251/final_project/annotations'
os.chdir(annot_path)

In [9]:
# parse xml to dataframe
def parse_xml(xml_file_name):
    tree = ET.parse(xml_file_name)  
    root = tree.getroot()
    lst=[]
    for elems in root:
        # print(elems.attrib)
            for subelem in elems:
                if subelem.attrib.get('label') != None:
                    subelem.attrib.update(id=elems.attrib.get('id'))
                    subelem.attrib.update(names=elems.attrib.get('name'))
                    subelem.attrib.update(width=elems.attrib.get('width'))
                    subelem.attrib.update(height=elems.attrib.get('height'))
                    lst.append(subelem.attrib)
    columns=subelem.attrib.keys()
    df=pd.DataFrame(lst,columns=columns)
    # df['names']=df.names.apply(lambda x: x.split('/')[1])
    return df


In [10]:
df_silas=parse_xml('construction_annotations_silas.xml')
df_sudha=parse_xml('constructions_annotations_sudha.xml')
df_andrew=parse_xml('construction_annotations_andrew.xml')
df=pd.concat([df_andrew,df_sudha,df_silas])

In [11]:
# standarize id
df=df.drop('id',axis=1)
# normalize file names remove '/'
df['names']=[name.split('/')[1] if '/' in name else name.split('/')[0] for name in df.names.values]
# add splits for train and val to df
uniq_names=sorted(list(set(df.names)))
size=len(uniq_names)
t_size=.8
v_size=1-t_size
index_split=['train']*int(np.round(size*t_size,0))+['val']*int(np.round(size*v_size,0))
shuffle(index_split)
df1=pd.DataFrame()
df1['id']=[i for i in range(df.names.unique().shape[0])]
df1['split']=index_split
df1['names']=uniq_names
df_split=df.merge(df1, on='names')
df_split=df_split[['label', 'xtl', 'ytl', 'xbr', 'ybr', 'occluded', 'id', 'names', 'width',
       'height', 'split']]


### quick look at df

In [ ]:
# most images have less than 5 annotations
df_split.groupby(['names'])['label'].agg({'count'}).hist(figsize=(10,5), xlabelsize=10, bins=20);


In [ ]:
# more people are labeled than safetyvest or hardhat
pd.DataFrame([df_split.label.astype('category').cat.codes,df_split.label]).T.groupby(['label']).count().plot(figsize=(10,5),kind='bar',rot=.40, legend=False)
# plot(kind='hist',index=['a','b'],xticks=None)

In [ ]:
# df_split.iloc[:,1:5].astype('float')

In [ ]:
# mean size (area) of annotations is quite small
bw=df_split.xbr.astype('float')-df_split.xtl.astype('float')
bh=df_split.ybr.astype('float')-df_split.ytl.astype('float')
df_l=pd.DataFrame({'label':df_split.label,'rel_area':bw*bh/(640*360)})
x,y=np.histogram(df_l.rel_area,bins=8)
df_l['cut']=pd.cut(df_l.rel_area,8)
df_l.groupby(['label','cut'])[['rel_area']].agg({'mean', 'count'})

In [ ]:
x_=df_split[df_split.label!='person'].xbr.astype('float')-df_split[df_split.label!='person'].xtl.astype('float')
y_=df_split[df_split.label!='person'].ybr.astype('float')-df_split[df_split.label!='person'].ytl.astype('float')
X=np.array([[x,y] for x,y in list(zip(x_,y_))])
kmeans = KMeans(n_clusters=5)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis_r')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.7);
plt.title("Hardhats & Safetyvests")
plt.xlabel("x-dimension")
plt.ylabel("y-dimension")
plt.show()

In [ ]:
# np.concatenate((centers[:3][:,0]/640,centers[:3][:,1]/360)).reshape(2,3).T

In [ ]:
d=df_split[df_split.label!='person']
d.insert(1,'clust', y_kmeans, True)
d.groupby('clust')['label'].count()

In [ ]:
# get min, max values of each bbx w safetyvest and hardhat label
lstx=[]
lsty=[]
for i in range(5):
    lstx.append((min(d[d.clust==i].xbr.astype('float')-d[d.clust==i].xtl.astype('float')),max(d[d.clust==i].xbr.astype('float')-d[d.clust==i].xtl.astype('float'))))
    lsty.append((min(d[d.clust==i].ybr.astype('float')-d[d.clust==i].ytl.astype('float')),max(d[d.clust==i].ybr.astype('float')-d[d.clust==i].ytl.astype('float'))))

In [ ]:
s=np.concatenate((np.array(lstx)[:,1],np.array(lsty)[:,1])).reshape(2,5).T

In [ ]:
x_=df_split[df_split.label=='person'].xbr.astype('float')-df_split[df_split.label=='person'].xtl.astype('float')
y_=df_split[df_split.label=='person'].ybr.astype('float')-df_split[df_split.label=='person'].ytl.astype('float')
X=np.array([[x,y] for x,y in list(zip(x_,y_))])
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis_r')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=100, alpha=0.7);
plt.title("Persons")
plt.xlabel("x-dimension")
plt.ylabel("y-dimension")
plt.show()

In [ ]:
# np.concatenate((centers[:][:,0]/640,centers[:][:,1]/360)).reshape(2,4).T

In [ ]:
d=df_split[df_split.label=='person']
d.insert(1,'clust', y_kmeans, True)
d.groupby('clust')['label'].count()

In [ ]:
# get min, max values of each bbx w person label
lstx=[]
lsty=[]
for i in range(4):
    lstx.append((min(d[d.clust==i].xbr.astype('float')-d[d.clust==i].xtl.astype('float')),max(d[d.clust==i].xbr.astype('float')-d[d.clust==i].xtl.astype('float'))))
    lsty.append((min(d[d.clust==i].ybr.astype('float')-d[d.clust==i].ytl.astype('float')),max(d[d.clust==i].ybr.astype('float')-d[d.clust==i].ytl.astype('float'))))

In [ ]:
t=np.concatenate((np.array(lstx)[:,1],np.array(lsty)[:,1])).reshape(2,4).T

In [ ]:
# max values of w,h of each bbox
arr=np.concatenate((s,t))
arr.sort(axis=0)
arr

### other formats

In [ ]:
# templates
kitti_template = "{label} 0.0 0 0.0 {xtl} {ytl} {xbr} {ybr} 0.0 0.0 0.0 0.0 0.0 0.0 0.0"
voc_pascal_template_outer= '  <image id="{}" name="{}" width="{}" height="{}">' 
voc_pascal_template_head='  <version>1.1</version>\n  <meta>\n    <task>\n      <id>1</id>\n      <name>ahl251</name>\n      <size>{}</size>\n      <mode>annotation</mode>\n      <overlap>0</overlap>\n      <bugtracker/>\n      <flipped>False</flipped>\n      <created>2019-03-30 23:40:47.695960+03:00</created>\n      <updated>2019-03-31 06:51:49.089135+03:00</updated>\n      <labels>\n        <label>\n          <name>person</name>\n          <attributes> </attributes>\n        </label>\n          <label>\n            <name>safetyvest</name>\n            <attributes> </attributes>\n        </label>\n          <label>\n            <name>hardhat</name>\n            <attributes> </attributes>\n        </label>\n          </labels>\n      <segments>\n        <segment>\n         <id>117</id>\n         <start>0</start>\n         <stop>{}</stop>\n         <url>http://c.onepanel.io/?id=117</url>\n        </segment>\n      </segments>\n      <owner>\n        <username>admin</username>\n        <email>admin@onepanel.io</email>\n      </owner>\n    </task>\n    <dumped>2019-03-31 06:51:51.787681+03:00</dumped>\n  </meta>'
voc_pascal_template_inner= '    <box label="{}" xtl="{}" ytl="{}" xbr="{}" ybr="{}" occluded="{}"> </box>'
template_test= '  <box label="{label}" xtl="{xtl}" ytl="{ytl}" xbr="{xbr}" ybr="{ybr}" occluded="{occluded}"> </box>'

# for yolo see below

### to TF format

In [ ]:
### need to add correct file names

In [ ]:
src_dir='all_images'
dir_name='tf_format'
dst_dir_img = 'tf_format/images/'
xml_file_name='construction_annotations.xml'
path=os.getcwd()
os.mkdir(os.path.join(path,dir_name))
os.mkdir(os.path.join(path,dst_dir_img))

label_codes={'hardhat':'0', 'person':'1', 'safetyvest':'2'}
tree = ET.parse(xml_file_name)  
root = tree.getroot()
labels=[]
for elems in root:
    lst=[]
    if type(elems.attrib.get('name'))=='NoneType':
        break
    elif '/' in elems.attrib.get('name'):
        try:
            name=elems.attrib.get('name').split('/')[1]
            lst.append(name)
            shutil.copy(os.path.join(src_dir, name), dst_dir_img) # 
        except:
            pass
    else:
        try:
            name=elems.attrib.get('name')
            lst.append(name)
            shutil.copy(os.path.join(src_dir, name), dst_dir_img) # 
        except:
            pass
        
    for subelem in elems:
        # print(subelem.attrib.get('label'))
        lab=subelem.attrib.get('label')
        
        if lab != None:
            lst+=list(subelem.attrib.values())[1:5]+[label_codes[lab]]
            labels.append(lab)
    with open(os.path.join(path, dir_name, name + ".txt"), "w") as text_file:
        text_file.write(" ".join(lst))
    
    # if a single file is needed with all data
    with open(os.path.join(path, dir_name, "labels.txt"), "a") as text_file:
        text_file.write(" ".join(lst+['\n']))
        
with open(os.path.join(path, dir_name, "classes.txt"), "w") as text_file:
    text_file.write("\n".join(set(labels)))

### voc_pascal

In [ ]:
df=df_split
src_dir='all_images/'
dst_dir_img = 'voc/images/'
dst_dir_labels = 'voc/labels/'
os.makedirs(dst_dir_img)
os.makedirs(dst_dir_labels)

xml_output=[]
img_names=sorted(list(set(df.names)))
for nm in img_names:
    
    shutil.copy(src_dir+nm, dst_dir_img)  # save valid images to voc dir
    
    header=list(df[df.names==nm].iloc[:,6:].values[0])
    xml_output.append(voc_pascal_template_outer.format(*header))
    rows=df[df.names==nm].iloc[:,:6].reset_index(drop=True)
    for idx in range(rows.T.shape[1]):
        row=list(rows.T[idx].values)
        xml_output.append(voc_pascal_template_inner.format(*row))
        if idx == rows.T.shape[1]-1:
            xml_output.append('  </image>')

with open(dst_dir_labels+"construction_annotations" + ".xml", "w") as text_file:
    text_file.write("<annotations>\n")
    text_file.write(voc_pascal_template_head.format(*[df.names.unique().shape[0]]*2))
    text_file.write("\n".join(xml_output))
    text_file.write("\n</annotations>")

shutil.make_archive('voc/'+'images', 'zip', dst_dir_img)

### kitti

In [ ]:
# create file structure
os.chdir('/home/silas/MIDS/W251/final_project/annotations/')
os.makedirs('data/train/images')
os.makedirs('data/train/labels')
os.makedirs('data/val/images')
os.makedirs('data/val/labels')


def apply_split_kitti(df,split_type):
    """
    Arg: split_type takes either 'train' or 'val'
    Creates separate .txt files for each img annotation using kitti format
    """
    split_type=split_type
    # get all val
    test_df=df_split[df_split.split==split_type]
    # get all unique names of images in val
    unique_lst=test_df.names.unique()
    # get a dataframe representing a unique name of image
    #tmp=df_test[df_test.names==img].reset_index(drop=True)
    for img in unique_lst:
    
        filename=img.split('.')[0]
    
        tmp=test_df[test_df.names==img].reset_index(drop=True)
    
        for i in range(tmp.shape[0]):
        
            inner=kitti_template.format(**tmp.loc[i].to_dict())
            completeName = os.path.join('data/{}/labels'.format(split_type), filename+".txt")
        
            with open(completeName, "a") as text_file:
                text_file.write(inner+'\n')
        
            src_dir='all_images/'
            dst_dir = "data/{}/images".format(split_type)
            shutil.copy(src_dir+img, dst_dir)

    
apply_split_kitti(df_split,'val')
apply_split_kitti(df_split,'train')

In [ ]:
"""
# directory and file structure kitti

all_images

data
    train
        image
            <name_of_image>.img
        label
            <name_of_label>.txt
    val
        image
            <name_of_image>.img
        label
            <name_of_label>.txt
""";

### yolo

In [12]:
def yolo_df(df):
    df_yolo=pd.DataFrame()
    df_yolo=df.iloc[:,1:5].astype('float')
    df_yolo['label']=df.label.astype('category').cat.codes
    df_yolo['name']=df.names
    df_yolo['label_']=df.label
    return df_yolo

In [13]:
def yolo_format(df_yolo):
    X1 = df_yolo['xtl']
    X2 = df_yolo['xbr']
    Y1 = df_yolo['ytl']
    Y2 = df_yolo['ybr']
    name=df_yolo.name
    return df_yolo.label, (X1+(X2-X1)/2)/640, (Y1+(Y2-Y1)/2)/360, (X2-X1)/640, (Y2-Y1)/360, name

In [20]:
num_train=int(len(img_lst)*.8)

In [23]:
path=os.getcwd()
os.makedirs(path+'/yolo/data/obj')
df_y=yolo_df(df_split)
obj_path=os.path.join(path, 'yolo','data', 'obj')


# make all img .txt files and store in obj directory
for un in df_y.name.unique():
    df_tmp=df_y[df_y.name==un].reset_index(drop=True)
    label,x,y,width,height,name=yolo_format(df_tmp)

    for item in list(zip(label,x,y,width,height)):
        row='{} {:2f} {:2f} {:2f} {:2f}'.format(item[0], item[1], item[2], item[3], item[4])
        path_txt= os.path.join(obj_path, un.replace('.jpeg','.txt'))
        
        with open(path_txt, "a") as text_file:
            text_file.write(row+'\n')
            
        src_dir='all_images/'
        shutil.copy(src_dir+un, obj_path) 

# shuffle train/valid
img_lst=[img for img in sorted([i for i in os.listdir(obj_path) if '.jpeg' in i])]
lst_train=img_lst[:num_train]
lst_valid=img_lst[num_train:]
        
# make train.txt
for img in sorted(lst_train):
    print(img)
    with open(os.path.join('yolo','data', 'train.list'), "a") as train_txt:
        train_txt.write(os.path.join('/root','darknet','yolo', 'obj', img)+'\n')
        # make valid.txt
for img in sorted(lst_valid):
    with open(os.path.join('yolo','data', 'valid.list'), "a") as train_txt:
        train_txt.write(os.path.join('/root','darknet','yolo', 'obj', img)+'\n')

# write obj.data file 
objs='classes={}\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = backup/'.format(df_y.label.nunique())

with open (os.path.join('yolo','data', 'obj.data'), "a") as obj_data:
    obj_data.write(objs) 

# make obj.names file 
for labl in sorted(df_y.label_.unique()):
    with open(os.path.join('yolo','data', 'obj.names'), "a") as text_file:
            text_file.write(labl+'\n')
    with open(os.path.join(obj_path, 'classes.txt'), "a") as text_file:
            text_file.write(labl+'\n')

Sequence01_0.jpeg
Sequence01_1.jpeg
Sequence01_10.jpeg
Sequence01_11.jpeg
Sequence01_12.jpeg
Sequence01_13.jpeg
Sequence01_14.jpeg
Sequence01_15.jpeg
Sequence01_16.jpeg
Sequence01_17.jpeg
Sequence01_18.jpeg
Sequence01_19.jpeg
Sequence01_2.jpeg
Sequence01_20.jpeg
Sequence01_21.jpeg
Sequence01_22.jpeg
Sequence01_23.jpeg
Sequence01_24.jpeg
Sequence01_25.jpeg
Sequence01_26.jpeg
Sequence01_27.jpeg
Sequence01_28.jpeg
Sequence01_29.jpeg
Sequence01_3.jpeg
Sequence01_30.jpeg
Sequence01_31.jpeg
Sequence01_32.jpeg
Sequence01_33.jpeg
Sequence01_34.jpeg
Sequence01_35.jpeg
Sequence01_36.jpeg
Sequence01_37.jpeg
Sequence01_38.jpeg
Sequence01_39.jpeg
Sequence01_4.jpeg
Sequence01_40.jpeg
Sequence01_41.jpeg
Sequence01_42.jpeg
Sequence01_43.jpeg
Sequence01_44.jpeg
Sequence01_45.jpeg
Sequence01_46.jpeg
Sequence01_47.jpeg
Sequence01_48.jpeg
Sequence01_49.jpeg
Sequence01_5.jpeg
Sequence01_50.jpeg
Sequence01_51.jpeg
Sequence01_52.jpeg
Sequence01_53.jpeg
Sequence01_54.jpeg
Sequence01_55.jpeg
Sequence01_56.jpeg

In [ ]:
"""
# yolo directory and file structure

data
    obj.names
        person
        hardhat
        safetyvest
    
    
    obj.data
       '''
       # write in file
        classes= 2
        train  = data/train.txt
        valid  = data/test.txt
        names = data/obj.names
        backup = backup/
       '''
        
    train.txt
        data/obj/1.jpg
        data/obj/2.jpg
       
        
    obj/
      1.jpg
      1.txt  
      2.jpg
      2.txt
"""   
    

### scraps to see bounding boxes on images

In [ ]:
df_see=df_split.iloc[:,1:5].astype(float)
xmin,ymin,xmax,ymax=df_see.loc[0].to_dict().values()

In [ ]:
# take an example
df_see['names']=df_split['names']
bb1=df_see.loc[0][4]
img=cv2.imread('all_images/{}'.format(bb1))

In [ ]:
rect=cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)),(100,0,0),1)

In [ ]:
cv2.imshow('dst_rt', rect)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# self draw bbox with cv2
r = cv2.selectROI(img)